# Part A. Extract Information

## 1. Crawling 9th Gen Pokemon from Wiki

Please ignore this code because the file is already stored in the data. The purpose of crawler is to match pokemons of 9th generation with other data to make analysis.

In [1]:
from lib.pokeStudy import Crawler

In [ ]:
link = "https://wiki.52poke.com/zh/%E5%AE%9D%E5%8F%AF%E6%A2%A6%E5%88%97%E8%A1%A8%EF%BC%88%E6%8C%89%E5%B8%95%E5%BA%95%E4%BA%9A%E5%9B%BE%E9%89%B4%E7%BC%96%E5%8F%B7%EF%BC%89"
crawler = Crawler()
df = crawler.download_9th_pokemon(link)

In [ ]:
df.to_csv("data\\poke9gen.csv")

## 2. Retrieve Information from Data

In [3]:
import pandas as pd

In [16]:
temp_df = pd.read_csv("data\\poke9gen.csv",index_col="ORI_DEX")

In [17]:
temp_df.head(3)

,Unnamed: 0,DEX,URL,CNT_NAME
ORI_DEX,,,,
1,0,906,/wiki/%E6%96%B0%E5%8F%B6%E5%96%B5,新叶喵
2,1,907,/wiki/%E8%92%82%E8%95%BE%E5%96%B5,蒂蕾喵
3,2,908,/wiki/%E9%AD%94%E5%B9%BB%E5%81%87%E9%9D%A2%E5%...,魔幻假面喵


In [19]:
temp_df = temp_df.drop("Unnamed: 0",axis=1)

In [35]:
temp_df.head(3)

,DEX,URL,CNT_NAME
ORI_DEX,,,
1,906,/wiki/%E6%96%B0%E5%8F%B6%E5%96%B5,新叶喵
2,907,/wiki/%E8%92%82%E8%95%BE%E5%96%B5,蒂蕾喵
3,908,/wiki/%E9%AD%94%E5%B9%BB%E5%81%87%E9%9D%A2%E5%...,魔幻假面喵


In [36]:
temp_df.tail(3)

,DEX,URL,CNT_NAME
ORI_DEX,,,
398,1006,/wiki/%E9%93%81%E6%AD%A6%E8%80%85,铁武者
399,1007,/wiki/%E6%95%85%E5%8B%92%E9%A1%BF,故勒顿
400,1008,/wiki/%E5%AF%86%E5%8B%92%E9%A1%BF,密勒顿


## 3. Get and Convert poke_info into DataFrame(df)

In [8]:
import re

In [6]:
with open("data\\poke_info.txt", "r", encoding='utf-8') as file:
    content = file.read()
    file.close()

In [10]:
is_name = False
names= []
stats = []
temp_stat = []
ind = -1
for each in re.split(r'\n', content):
    if each != "======":
        if is_name:
            names.append(each)
            stats.append([])
            ind+=1
        else:
            stats[ind].append(each)
    else:
        is_name = not is_name

In [13]:
print("Are lengths equal? %s"%(len(names) == len(stats)))

Are lengths equal? True


The next goal is to polish up the data structure of names and stats

### 3.1 Polish up (names)

In [15]:
mem = {}
for name in names:
    length = len(name.split(" "))
    if length in mem:
        mem[length].append(name)
    else:
        mem[length] = []

print(len(mem[5]))
print(len(mem[6]))
print(len(mem[7]))
print(mem[5][0])
print(mem[6][0])
print(mem[7][1])

144
507
13
005 - Charmeleon (Stage: 2)
1019 - Pikachu-1 #074 (Stage: 2)
981 - Sandy Shocks #381 (Stage: 3)


In [98]:
mem[6][0].split(" ")[-1][:-1]

'2'

In [33]:
int(mem[6][0].split(" ")[3][1:])

74

In [34]:
int(mem[7][0].split(" ")[4][1:])

378

### 3.2 Polish up (stats)

In [46]:
stats[0][0].split(" ")[2].split(".")

['39', '52', '43', '60', '50', '65']

In [52]:
for each in stats[:10]:
    print(each[5][6:].split(" / "))

['Fire']
['Fire']
['Fire', 'Flying']
['Electric']
['Electric']
['Electric']
['Electric']
['Electric']
['Electric']
['Electric']


In [77]:
columns = [
    "index",
    "name",
    "ch_name",
    "hp",
    "attack",
    "defense",
    "sp_attack",
    "sp_defense",
    "speed",
    "attr_1",
    "attr_2"
]

In [90]:
indexes = set()
result = pd.DataFrame(columns=columns)
for i, name in enumerate(names):
    # split each name with space
    temp = name.split(" ")
    length = len(name.split(" "))
    
    # if length == 6, third+1 val is the index
    # if length == 7, fourth+1 val is the index
    # Only 5, 6, 7 are lengths
    if length == 5:
        continue
    
    index = 0
    name =""
    if length == 6:
        name = temp[2]
        index = int(temp[3][1:])
    elif length == 7:
        name = temp[2]+temp[3]
        index = int(temp[4][1:])
        indexes.add(index)   
        
    if index in indexes:
        continue
    
    # index, name, ch_name
    indexes.add(index)
    ch_name = temp_df.iloc[index-1]["CNT_NAME"] 
    
    stat = stats[i][0].split(" ")[2].split(".") 
    hp = int(stat[0])
    attack = int(stat[1])
    defense = int(stat[2])
    sp_attack = int(stat[3])
    sp_defense = int(stat[4])
    speed = int(stat[5])
    attrs = stats[i][5][6:].split(" / ")
    attr_1 = attrs[0]
    attr_2 = attrs[0]
    if len(attrs) == 2:
        attr_2 = attrs[1]
    
    
    series = pd.Series({
    "index":index,
    "name":name,
    "ch_name":ch_name,
    "hp":hp,
    "attack":attack,
    "defense":defense,
    "sp_attack":sp_attack,
    "sp_defense":sp_defense,
    "speed":speed,
    "attr_1":attr_1,
    "attr_2":attr_2
    })
    
    result = result.append(series, ignore_index=True)

## 4. Save to csv

In [91]:
result.to_csv("output\\poke_info.csv")

# Part B. Data Analysis

In [101]:
result.sort_values("speed", axis=0, ascending=False).head(10)

,index,name,ch_name,hp,attack,defense,sp_attack,sp_defense,speed,attr_1,attr_2
30,208,Electrode,顽皮雷弹,60,50,70,80,80,150,Electric,Electric
296,307,Dragapult,多龙巴鲁托,88,120,75,100,75,142,Dragon,Ghost
274,137,Barraskewda,戽斗尖梭,61,123,60,60,50,136,Water,Water
370,394,Chien-Pao,古剑豹,80,120,80,90,65,135,Dark,Ice
374,400,Miraidon,密勒顿,100,85,100,135,115,135,Electric,Dragon
373,399,Koraidon,故勒顿,100,135,115,85,100,135,Fighting,Dragon
39,181,Jolteon,雷伊布,65,65,60,110,95,130,Electric,Electric
207,21,Talonflame,烈箭鹰,78,81,71,74,69,126,Fire,Flying
164,231,Weavile,玛狃拉,70,120,65,45,85,125,Dark,Ice
349,178,Kilowattrel,大电海燕,70,70,60,105,60,125,Electric,Flying


In [102]:
result.sort_values("attack", axis=0, ascending=False).head(10)

,index,name,ch_name,hp,attack,defense,sp_attack,sp_defense,speed,attr_1,attr_2
94,80,Slaking,请假王,150,160,100,95,65,100,Normal,Normal
192,157,Haxorus,双斧战龙,76,147,90,60,70,97,Dragon,Dragon
342,390,Baxcalibur,戟脊龙,115,145,92,75,86,87,Dragon,Ice
124,278,Salamence,暴飞龙,95,135,80,110,80,100,Dragon,Flying
383,369,Kingambit,仆刀将军,100,135,120,60,85,50,Dark,Steel
373,399,Koraidon,故勒顿,100,135,115,85,100,135,Fighting,Dragon
43,349,Dragonite,快龙,91,134,95,100,100,80,Dragon,Flying
82,318,Tyranitar,班基拉斯,100,134,110,95,100,61,Rock,Dark
236,119,Crabominable,好胜毛蟹,97,132,77,62,67,43,Fighting,Ice
194,356,Beartic,冻原熊,95,130,80,70,80,50,Ice,Ice


In [103]:
result.sort_values("sp_attack", axis=0, ascending=False).head(10)

,index,name,ch_name,hp,attack,defense,sp_attack,sp_defense,speed,attr_1,attr_2
281,284,Hatterene,布莉姆温,57,90,95,136,103,29,Psychic,Fairy
372,396,Chi-Yu,古玉鱼,55,80,80,135,120,100,Dark,Fire
374,400,Miraidon,密勒顿,100,85,100,135,115,135,Electric,Dragon
204,275,Volcarona,火神蛾,85,60,65,135,105,100,Bug,Fire
278,238,Polteageist,怖思壶,60,65,65,134,114,70,Ghost,Ghost
368,392,Gholdengo,赛富豪,87,60,95,133,91,84,Steel,Ghost
58,183,Espeon,太阳伊布,65,65,60,130,95,110,Psychic,Psychic
26,70,Gengar,耿鬼,60,65,60,130,75,110,Ghost,Poison
167,186,Glaceon,冰伊布,65,60,110,130,95,65,Ice,Ice
165,211,Magnezone,自爆磁怪,70,70,115,130,90,60,Electric,Steel


In [104]:
result.sort_values("hp", axis=0, ascending=False).head(10)

,index,name,ch_name,hp,attack,defense,sp_attack,sp_defense,speed,attr_1,attr_2
79,45,Blissey,幸福蛋,255,10,10,75,135,55,Normal,Normal
31,44,Chansey,吉利蛋,250,5,5,35,105,50,Normal,Normal
339,362,Cetitan,浩大鲸,170,113,65,45,55,73,Ice,Ice
186,336,Alomomola,保姆曼波,165,75,80,40,45,65,Water,Water
369,395,Ting-Lu,古鼎鹿,155,110,125,55,80,45,Dark,Ground
322,374,Dondozo,吃吼霸,150,100,115,65,65,35,Water,Water
94,80,Slaking,请假王,150,160,100,95,65,100,Normal,Normal
141,144,Drifblim,随风球,150,80,44,90,54,80,Ghost,Flying
96,117,Hariyama,铁掌力士,144,120,60,40,60,50,Fighting,Fighting
3,61,Wigglytuff,胖可丁,140,70,45,85,50,45,Normal,Fairy
